In [ ]:
# ------------------------------ Install Chroma and Google Gen AI ------------------------------
#!pip install chromadb
#!pip install google-generativeai

In [ ]:
# ------------------------------ Setup ------------------------------

import os
import chromadb.utils.embedding_functions as embedding_functions
import chromadb
import google.generativeai as genai
from google.colab import userdata

GOOGLE_API_KEY = userdata.get("GOOGLE_API_KEY")

genai.configure(api_key=GOOGLE_API_KEY)

google_ef = embedding_functions.GoogleGenerativeAiEmbeddingFunction(api_key=GOOGLE_API_KEY)

model = genai.GenerativeModel("gemini-pro")

In [ ]:
# ------------------------------ Functions and template ------------------------------

for m in genai.list_models():
    if "embedContent" in m.supported_generation_methods:
        print(m.name)

def build_prompt(query, context):
    base_prompt = {
        "content": "Ti faccio una domanda alla quale vorrei che tu rispondessi solamente tramite una query SQL per un database PostgreSQL."
        "basato solo sulla DDL fornita qui di seguito e il contesto in input."
        "Se non ci sono abbastanza informazioni nel contesto per rispondere alla domanda,"
        """"
        --
-- PostgreSQL database dump
--

-- Dumped from database version 9.6.24
-- Dumped by pg_dump version 9.6.24

SET statement_timeout = 0;
SET lock_timeout = 0;
SET idle_in_transaction_session_timeout = 0;
SET client_encoding = 'UTF8';
SET standard_conforming_strings = on;
SELECT pg_catalog.set_config('search_path', '', false);
SET check_function_bodies = false;
SET xmloption = content;
SET client_min_messages = warning;
SET row_security = off;

SET default_tablespace = '';

SET default_with_oids = false;

--
-- Name: alert; Type: TABLE; Schema: public; Owner: postgres
--

CREATE TABLE public.alert (
    id bigint NOT NULL,
    id_topic integer NOT NULL,
    trigger_time timestamp with time zone DEFAULT now() NOT NULL,
    expired_time timestamp with time zone,
    pkt_uid text NOT NULL,
    ae_value double precision NOT NULL
);


ALTER TABLE public.alert OWNER TO postgres;

--
-- Name: alert_id_seq; Type: SEQUENCE; Schema: public; Owner: postgres
--

CREATE SEQUENCE public.alert_id_seq
    START WITH 1
    INCREMENT BY 1
    NO MINVALUE
    NO MAXVALUE
    CACHE 1;


ALTER TABLE public.alert_id_seq OWNER TO postgres;

--
-- Name: alert_id_seq; Type: SEQUENCE OWNED BY; Schema: public; Owner: postgres
--

ALTER SEQUENCE public.alert_id_seq OWNED BY public.alert.id;


--
-- Name: alert_severity; Type: TABLE; Schema: public; Owner: postgres
--

CREATE TABLE public.alert_severity (
    id smallint NOT NULL,
    severity smallint NOT NULL,
    label character varying(30) NOT NULL,
    display_color_hex character varying(7) NOT NULL,
    threshold numeric(18,3)
);


ALTER TABLE public.alert_severity OWNER TO postgres;

--
-- Name: alert_state_id_seq; Type: SEQUENCE; Schema: public; Owner: postgres
--

CREATE SEQUENCE public.alert_state_id_seq
    START WITH 1
    INCREMENT BY 1
    NO MINVALUE
    NO MAXVALUE
    CACHE 1;


ALTER TABLE public.alert_state_id_seq OWNER TO postgres;

--
-- Name: alert_state_id_seq; Type: SEQUENCE OWNED BY; Schema: public; Owner: postgres
--

ALTER SEQUENCE public.alert_state_id_seq OWNED BY public.alert_severity.id;


--
-- Name: alert_work; Type: TABLE; Schema: public; Owner: postgres
--

CREATE TABLE public.alert_work (
    id bigint NOT NULL,
    id_alert bigint NOT NULL,
    evt_time timestamp with time zone DEFAULT now() NOT NULL,
    evt_type_id smallint NOT NULL,
    notes text
);


ALTER TABLE public.alert_work OWNER TO postgres;

--
-- Name: alert_work_id_seq; Type: SEQUENCE; Schema: public; Owner: postgres
--

CREATE SEQUENCE public.alert_work_id_seq
    START WITH 1
    INCREMENT BY 1
    NO MINVALUE
    NO MAXVALUE
    CACHE 1;


ALTER TABLE public.alert_work_id_seq OWNER TO postgres;

--
-- Name: alert_work_id_seq; Type: SEQUENCE OWNED BY; Schema: public; Owner: postgres
--

ALTER SEQUENCE public.alert_work_id_seq OWNED BY public.alert_work.id;


--
-- Name: customer; Type: TABLE; Schema: public; Owner: postgres
--

CREATE TABLE public.customer (
    id integer DEFAULT nextval(('public.customer_id_seq'::text)::regclass) NOT NULL,
    label character varying(30) NOT NULL
);


ALTER TABLE public.customer OWNER TO postgres;

--
-- Name: sensor; Type: TABLE; Schema: public; Owner: postgres
--

CREATE TABLE public.sensor (
    id integer NOT NULL,
    site_id integer NOT NULL,
    topic text NOT NULL,
    label character varying(30),
    alias character varying(30)
);


ALTER TABLE public.sensor OWNER TO postgres;

--
-- Name: sensor_id_seq; Type: SEQUENCE; Schema: public; Owner: postgres
--

CREATE SEQUENCE public.sensor_id_seq
    START WITH 1
    INCREMENT BY 1
    NO MINVALUE
    NO MAXVALUE
    CACHE 1;


ALTER TABLE public.sensor_id_seq OWNER TO postgres;

--
-- Name: sensor_id_seq; Type: SEQUENCE OWNED BY; Schema: public; Owner: postgres
--

ALTER SEQUENCE public.sensor_id_seq OWNED BY public.sensor.id;


--
-- Name: site; Type: TABLE; Schema: public; Owner: postgres
--

CREATE TABLE public.site (
    id integer NOT NULL,
    id_customer integer NOT NULL,
    label character varying(64) NOT NULL,
    location_address character varying(64),
    location_city character varying(30),
    location_zip character varying(12),
    location_province character varying(24),
    location_country character varying(2) DEFAULT 'IT'::character varying,
    location_coordinate point
);


ALTER TABLE public.site OWNER TO postgres;

--
-- Name: site_id_seq; Type: SEQUENCE; Schema: public; Owner: postgres
--

CREATE SEQUENCE public.site_id_seq
    START WITH 1
    INCREMENT BY 1
    NO MINVALUE
    NO MAXVALUE
    CACHE 1;


ALTER TABLE public.site_id_seq OWNER TO postgres;

--
-- Name: site_id_seq; Type: SEQUENCE OWNED BY; Schema: public; Owner: postgres
--

ALTER SEQUENCE public.site_id_seq OWNED BY public.site.id;


--
-- Name: alert id; Type: DEFAULT; Schema: public; Owner: postgres
--

ALTER TABLE ONLY public.alert ALTER COLUMN id SET DEFAULT nextval('public.alert_id_seq'::regclass);


--
-- Name: alert_severity id; Type: DEFAULT; Schema: public; Owner: postgres
--

ALTER TABLE ONLY public.alert_severity ALTER COLUMN id SET DEFAULT nextval('public.alert_state_id_seq'::regclass);


--
-- Name: alert_work id; Type: DEFAULT; Schema: public; Owner: postgres
--

ALTER TABLE ONLY public.alert_work ALTER COLUMN id SET DEFAULT nextval('public.alert_work_id_seq'::regclass);


--
-- Name: sensor id; Type: DEFAULT; Schema: public; Owner: postgres
--

ALTER TABLE ONLY public.sensor ALTER COLUMN id SET DEFAULT nextval('public.sensor_id_seq'::regclass);


--
-- Name: site id; Type: DEFAULT; Schema: public; Owner: postgres
--

ALTER TABLE ONLY public.site ALTER COLUMN id SET DEFAULT nextval('public.site_id_seq'::regclass);


--
-- Data for Name: alert; Type: TABLE DATA; Schema: public; Owner: postgres
--

INSERT INTO public.alert (id, id_topic, trigger_time, expired_time, pkt_uid, ae_value) VALUES (1, 5, '2022-10-14 09:58:25.211802+02', NULL, 'cd4hav0uke1r4evog9d0', 0.317388000000000003);


--
-- Name: alert_id_seq; Type: SEQUENCE SET; Schema: public; Owner: postgres
--

SELECT pg_catalog.setval('public.alert_id_seq', 1, true);


--
-- Data for Name: alert_severity; Type: TABLE DATA; Schema: public; Owner: postgres
--

INSERT INTO public.alert_severity (id, severity, label, display_color_hex, threshold) VALUES (1, 10, 'GIALLO', '#ffcc00', 0.550);
INSERT INTO public.alert_severity (id, severity, label, display_color_hex, threshold) VALUES (3, 1000, 'ROSSO', '#cc3300', 0.400);
INSERT INTO public.alert_severity (id, severity, label, display_color_hex, threshold) VALUES (2, 100, 'ARANCIO', '#ff9966', 0.450);


--
-- Name: alert_state_id_seq; Type: SEQUENCE SET; Schema: public; Owner: postgres
--

SELECT pg_catalog.setval('public.alert_state_id_seq', 3, true);


--
-- Data for Name: alert_work; Type: TABLE DATA; Schema: public; Owner: postgres
--

INSERT INTO public.alert_work (id, id_alert, evt_time, evt_type_id, notes) VALUES (3, 1, '2022-10-14 11:31:54.375525+02', 33, 'est');


--
-- Name: alert_work_id_seq; Type: SEQUENCE SET; Schema: public; Owner: postgres
--

SELECT pg_catalog.setval('public.alert_work_id_seq', 3, true);


--
-- Data for Name: customer; Type: TABLE DATA; Schema: public; Owner: postgres
--

INSERT INTO public.customer (id, label) VALUES (1, 'SAE');


--
-- Data for Name: sensor; Type: TABLE DATA; Schema: public; Owner: postgres
--

INSERT INTO public.sensor (id, site_id, topic, label, alias) VALUES (3, 3, '/sensors/delta/lab03/9ca525849f27-00', 'Sensore 01 LF', NULL);
INSERT INTO public.sensor (id, site_id, topic, label, alias) VALUES (2, 3, '/sensors/delta/lab03/9ca525849f35-00', 'Sensore 01 HF', NULL);
INSERT INTO public.sensor (id, site_id, topic, label, alias) VALUES (1, 3, '/sensors/delta/lab03/0050c2de0007-00', 'Sensore Test 101', NULL);
INSERT INTO public.sensor (id, site_id, topic, label, alias) VALUES (5, 3, '/sensors/delta/lab03/0050c2de0030-00', 'Sensore Test 102', NULL);
INSERT INTO public.sensor (id, site_id, topic, label, alias) VALUES (6, 3, '/sensors/delta/lab03/0050c2de0032-00', 'Sensore Test 103', NULL);
INSERT INTO public.sensor (id, site_id, topic, label, alias) VALUES (4, 3, '/sensors/delta/lab03/0050c2de0035-00', 'Sensore Test 104', NULL);


--
-- Name: sensor_id_seq; Type: SEQUENCE SET; Schema: public; Owner: postgres
--

SELECT pg_catalog.setval('public.sensor_id_seq', 6, true);


--
-- Data for Name: site; Type: TABLE DATA; Schema: public; Owner: postgres
--

INSERT INTO public.site (id, id_customer, label, location_address, location_city, location_zip, location_province, location_country, location_coordinate) VALUES (3, 1, 'LAB03', NULL, 'Roma', '00100', 'RM', 'IT', NULL);


--
-- Name: site_id_seq; Type: SEQUENCE SET; Schema: public; Owner: postgres
--

SELECT pg_catalog.setval('public.site_id_seq', 3, true);


--
-- Name: alert alert_pkey; Type: CONSTRAINT; Schema: public; Owner: postgres
--

ALTER TABLE ONLY public.alert
    ADD CONSTRAINT alert_pkey PRIMARY KEY (id);


--
-- Name: alert_severity alert_severity_pkey; Type: CONSTRAINT; Schema: public; Owner: postgres
--

ALTER TABLE ONLY public.alert_severity
    ADD CONSTRAINT alert_severity_pkey PRIMARY KEY (id);


--
-- Name: alert_work alert_work_pkey; Type: CONSTRAINT; Schema: public; Owner: postgres
--

ALTER TABLE ONLY public.alert_work
    ADD CONSTRAINT alert_work_pkey PRIMARY KEY (id);


--
-- Name: customer customer_pkey; Type: CONSTRAINT; Schema: public; Owner: postgres
--

ALTER TABLE ONLY public.customer
    ADD CONSTRAINT customer_pkey PRIMARY KEY (id);


--
-- Name: alert idx_alert-trigger_time_pkt_uid; Type: CONSTRAINT; Schema: public; Owner: postgres
--

ALTER TABLE ONLY public.alert
    ADD CONSTRAINT "idx_alert-trigger_time_pkt_uid" UNIQUE (trigger_time, pkt_uid);


--
-- Name: sensor idx_sensor-topic; Type: CONSTRAINT; Schema: public; Owner: postgres
--

ALTER TABLE ONLY public.sensor
    ADD CONSTRAINT "idx_sensor-topic" UNIQUE (topic);


--
-- Name: site idx_site-id_customer; Type: CONSTRAINT; Schema: public; Owner: postgres
--

ALTER TABLE ONLY public.site
    ADD CONSTRAINT "idx_site-id_customer" UNIQUE (id_customer) WITH (fillfactor='100');


--
-- Name: sensor sensor_pkey; Type: CONSTRAINT; Schema: public; Owner: postgres
--

ALTER TABLE ONLY public.sensor
    ADD CONSTRAINT sensor_pkey PRIMARY KEY (id);


--
-- Name: site site_pkey; Type: CONSTRAINT; Schema: public; Owner: postgres
--

ALTER TABLE ONLY public.site
    ADD CONSTRAINT site_pkey PRIMARY KEY (id);


--
-- Name: idx_alert-expired_time; Type: INDEX; Schema: public; Owner: postgres
--

CREATE INDEX "idx_alert-expired_time" ON public.alert USING btree (expired_time) WITH (fillfactor='100');


--
-- Name: idx_alert-id_topic; Type: INDEX; Schema: public; Owner: postgres
--

CREATE INDEX "idx_alert-id_topic" ON public.alert USING btree (id_topic);


--
-- Name: idx_alert_work-evt_time; Type: INDEX; Schema: public; Owner: postgres
--

CREATE INDEX "idx_alert_work-evt_time" ON public.alert_work USING btree (evt_time);


--
-- Name: idx_alert_work-evt_type_id; Type: INDEX; Schema: public; Owner: postgres
--

CREATE INDEX "idx_alert_work-evt_type_id" ON public.alert_work USING btree (evt_type_id);


--
-- Name: idx_alert_work-id_alert; Type: INDEX; Schema: public; Owner: postgres
--

CREATE INDEX "idx_alert_work-id_alert" ON public.alert_work USING btree (id_alert);


--
-- Name: idx_customer-label; Type: INDEX; Schema: public; Owner: postgres
--

CREATE UNIQUE INDEX "idx_customer-label" ON public.customer USING btree (label);


--
-- Name: idx_sensor-id_site; Type: INDEX; Schema: public; Owner: postgres
--

CREATE INDEX "idx_sensor-id_site" ON public.sensor USING btree (site_id) WITH (fillfactor='100');


--
-- Name: alert_work fk_alert_work-alert_id; Type: FK CONSTRAINT; Schema: public; Owner: postgres
--

ALTER TABLE ONLY public.alert_work
    ADD CONSTRAINT "fk_alert_work-alert_id" FOREIGN KEY (id_alert) REFERENCES public.alert(id) ON UPDATE CASCADE ON DELETE RESTRICT;


--
-- Name: sensor fk_sensor-site_id_site-id; Type: FK CONSTRAINT; Schema: public; Owner: postgres
--

ALTER TABLE ONLY public.sensor
    ADD CONSTRAINT "fk_sensor-site_id_site-id" FOREIGN KEY (site_id) REFERENCES public.site(id) ON UPDATE CASCADE ON DELETE RESTRICT;


--
-- Name: site fk_site-id_customer_customer_id; Type: FK CONSTRAINT; Schema: public; Owner: postgres
--

ALTER TABLE ONLY public.site
    ADD CONSTRAINT "fk_site-id_customer_customer_id" FOREIGN KEY (id_customer) REFERENCES public.customer(id) ON UPDATE CASCADE ON DELETE RESTRICT;


--
-- Name: TABLE customer; Type: ACL; Schema: public; Owner: postgres
--

GRANT SELECT ON TABLE public.customer TO grafreader;


--
-- Name: TABLE sensor; Type: ACL; Schema: public; Owner: postgres
--

GRANT SELECT ON TABLE public.sensor TO grafreader;


--
-- Name: TABLE site; Type: ACL; Schema: public; Owner: postgres
--

GRANT SELECT ON TABLE public.site TO grafreader;


--
-- PostgreSQL database dump complete
--



--
-- PostgreSQL database dump
--

-- Dumped from database version 9.6.24
-- Dumped by pg_dump version 9.6.24

SET statement_timeout = 0;
SET lock_timeout = 0;
SET idle_in_transaction_session_timeout = 0;
SET client_encoding = 'UTF8';
SET standard_conforming_strings = on;
SELECT pg_catalog.set_config('search_path', '', false);
SET check_function_bodies = false;
SET xmloption = content;
SET client_min_messages = warning;
SET row_security = off;

SET default_tablespace = '';

SET default_with_oids = false;

--
-- Name: measurement; Type: TABLE; Schema: public; Owner: postgres
--

CREATE TABLE public.measurement (
    pkt_time timestamp with time zone NOT NULL,
    sndr_host text,
    id_topic integer,
    pkt_uid text NOT NULL,
    pkt_metric json,
    pkt_mode smallint,
    ae_value double precision
);


ALTER TABLE public.measurement OWNER TO postgres;

--
-- Name: idx_measurement-pkt_time; Type: INDEX; Schema: public; Owner: postgres
--

CREATE INDEX "idx_measurement-pkt_time" ON public.measurement USING btree (pkt_time DESC);


--
-- Name: idx_measurement-pkt_time_pkt_uid; Type: INDEX; Schema: public; Owner: postgres
--

CREATE UNIQUE INDEX "idx_measurement-pkt_time_pkt_uid" ON public.measurement USING btree (pkt_time DESC, pkt_uid DESC);


--
-- Name: measurement ts_insert_blocker; Type: TRIGGER; Schema: public; Owner: postgres
--



--
-- Name: measurement fk_measure-id_topic_sensor_id; Type: FK CONSTRAINT; Schema: public; Owner: postgres
--

ALTER TABLE ONLY public.measurement
    ADD CONSTRAINT "fk_measure-id_topic_sensor_id" FOREIGN KEY (id_topic) REFERENCES public.sensor(id) ON UPDATE CASCADE ON DELETE RESTRICT;


--
-- Name: TABLE measurement; Type: ACL; Schema: public; Owner: postgres
--

GRANT SELECT ON TABLE public.measurement TO grafreader;


--
-- PostgreSQL database dump complete
--

--View: public.measure_last30m

CREATE VIEW public.measure_last30m
(
  sampletime,
  ae_value,
  sensor
)
AS
SELECT date_part('epoch'::text, ((inrows.metric ->>
    'readTime'::text))::timestamp with time zone) AS sampletime,
    ((inrows.metric ->> 'ae_value'::text))::double precision AS ae_value,
    inrows.id_topic AS sensor
FROM (
    SELECT json_array_elements(measurement.pkt_metric) AS metric,
        measurement.id_topic
    FROM measurement
    WHERE (measurement.pkt_time >= (now() - '00:30:00'::interval))
    ) inrows
ORDER BY (date_part('epoch'::text, ((inrows.metric ->>
    'readTime'::text))::timestamp with time zone));

ALTER TABLE public.measure_last30m
  OWNER TO postgres;

GRANT SELECT, INSERT, UPDATE, DELETE, REFERENCES, TRIGGER, TRUNCATE
  ON public.measure_last30m
TO postgres;

GRANT SELECT
  ON public.measure_last30m
TO grafreader;

--Function: public.record_alert()

CREATE OR REPLACE FUNCTION record_alert()
  RETURNS TRIGGER
  LANGUAGE PLPGSQL
  AS
$$
BEGIN
  IF NEW.ae_value < 0.55 THEN
    INSERT INTO alert(id_topic, pkt_uid, ae_value)
    VALUES(NEW.id_topic, NEW.pkt_uid, NEW.ae_value);
  END IF;

	RETURN NEW;
END;
$$

--Trigger: public.measurement.check_aevalue_for_alert

CREATE TRIGGER check_aevalue_for_alert
  AFTER INSERT
  ON public.measurement
  FOR EACH ROW
  EXECUTE PROCEDURE public.record_alert();


  """

        ,
    }
    user_prompt = {
        "content": f" La domanda è '{query}'. Ecco tutto il contesto che hai:"
        f'{(" ").join(context)}',
    }

    # combine the prompts to output a single prompt string
    system = f"{base_prompt['content']} {user_prompt['content']}"

    return system


def get_gemini_response(query, context):
    response = model.generate_content(build_prompt(query, context))
    return response.text



models/embedding-001


In [ ]:
# ------------------------------ Create Chroma Client and Collection ------------------------------


# Create a client
client = chromadb.Client()

it_collection = client.get_or_create_collection(name="it_coll", embedding_function=google_ef)


# Add documents
it_collection.add(
    documents=[
      """"

SELECT
  time_bucket('1 minute', pkt_time) as bucket,
  id_topic,
  avg(ae_value) as avg,
  max(ae_value) as max,
  min(ae_value) as min
FROM
  measurement
where id_topic = 6
GROUP BY bucket, id_topic
order by bucket;


    """

    ,

    """

select
	tm tm,
	knt,
	ae,
	case
		when ae >=0 and ae <=0.40 then 0
		when ae >0.40 and ae <=0.55 then 1
		when ae >0.55 and ae <=1.00 then 2 END
	as ae_cl,
	translate(translate(rv,'[',''),']','') rv
from
(
	select
	pkt_time::timestamp tm,
	ae_value ae,
	(select jsonb_agg(t -> 'rv')::text from json_array_elements(pkt_metric) as x(t)) as rv,
	jsonb_array_length((select jsonb_agg(t -> 'rv')::jsonb from json_array_elements(pkt_metric) as x(t))) knt
	from measurement
	where id_topic = 1 and ae_value >=0 and ae_value <> 'NaN'::NUMERIC
	limit 100
) r_v

    """
    ,
    """
        select
      pkt_time,
      pkt_metric,
      (pkt_metric::json -> 0)::json ->> 'rv' rv0000,
      (pkt_metric::json -> 1)::json ->> 'rv' rv0001,
      (pkt_metric::json -> 2)::json ->> 'rv' rv0002,
      (pkt_metric::json -> 3)::json ->> 'rv' rv0003,
      (pkt_metric::json -> 4)::json ->> 'rv' rv0004,
      (pkt_metric::json -> 5)::json ->> 'rv' rv0005,
      (pkt_metric::json -> 6)::json ->> 'rv' rv0006,
      (pkt_metric::json -> 7)::json ->> 'rv' rv0006,
      (pkt_metric::json -> 8)::json ->> 'rv' rv0007,
      (pkt_metric::json -> 8)::json ->> 'rv' rv0008,
      (pkt_metric::json -> 9)::json ->> 'rv' rv0009,

      (pkt_metric::json -> 1499)::json ->> 'rv' rv1499,
      ae_value
    from measurement
    where id_topic = 3
    limit 5

    """
    ,
    """

    SELECT tm, avg_value
    FROM
    (
    SELECT
      time_bucket('5 minute', ms.pkt_time) as tm,
      avg(ms.ae_value) avg_value
    FROM
      public.measurement as ms
    JOIN public.sensor as sr
    ON ms.id_topic=sr.id
    where sr.id = 6
    GROUP BY tm, sr.id
    HAVING avg(ms.ae_value) IS NOT NULL
    order by tm
    ) MY_Q
    WHERE avg_value IS NULL
    """
     ]
    ,
    metadatas=[{"source": "my_source"}, {"source": "my_source"}, {"source": "my_source"}, {"source": "my_source"}],
    ids=["id1", "id2", "id3", "id4"],
)

# Query
it_collection = client.get_collection(name="it_coll", embedding_function=google_ef)


In [ ]:
# ------------------------------ Query and results ------------------------------

# Successful querys
query = "Mostrami per il sensore 5 tutti gli alert"

# Query the collection to get the 2 most relevant results
results = it_collection.query(query_texts=[query], n_results=1, include=["documents", "metadatas"])


# Get the response from Gemini
response = get_gemini_response(query, results["documents"][0])

# Output, with sources
print(response)


```sql
SELECT 
    a.trigger_time,
    a.ae_value
FROM 
    alert a
JOIN 
    sensor s ON a.id_topic = s.id
WHERE 
    s.id = 5;
```


Risultati:

1) Mostrami tutti i sensori ✅

SELECT *
FROM sensor;

2) Mostrami i sensori con valore di AE maggiore di 0.55 ❌ Commento: query non ha dato errore, ma non ha dato risultati

SELECT DISTINCT s.id, s.label , s.alias
FROM sensor s
JOIN measurement m ON s.id = m.id_topic
JOIN alert a ON m.id_topic = a.id_topic
WHERE a.ae_value > 0.55;

3) Mostrami per il sensore 1, 100 elementi, con timestamp, knt, ae, ae_cl, dove ae_cl è 0, 1 o 2 per valori compresi rispettivamente tra 0 e 0.4, 0.4 e 0.55, 0.55 e 1.0, e i valori rv ✅


SELECT
    tm,
    knt,
    ae,
    CASE
        WHEN ae >= 0 AND ae <= 0.40 THEN 0
        WHEN ae > 0.40 AND ae <= 0.55 THEN 1
        WHEN ae > 0.55 AND ae <= 1.00 THEN 2
    END AS ae_cl,
    TRANSLATE(TRANSLATE(rv, '[', ''), ']', '') AS rv
FROM (
    SELECT
        pkt_time::TIMESTAMP tm,
        ae_value ae,
        (
            SELECT jsonb_agg(t ->> 'rv')::TEXT
            FROM json_array_elements(pkt_metric) AS x(t)
        ) AS rv,
        jsonb_array_length(
            (
                SELECT jsonb_agg(t ->> 'rv')::JSONB
                FROM json_array_elements(pkt_metric) AS x(t)
            )
        ) knt
    FROM
        measurement
    WHERE
        id_topic = 1
        AND ae_value >= 0
        AND ae_value <> 'NaN'::NUMERIC
    LIMIT 100
) r_v;

4) Per il sensore 3 mostrami la media, la minima e la massima del valore ae, su un intervallo di 3 minuti ✅ Commento: Ha fatto lui una query diversa da quella del DB, ma comunque corretta. Unica cosa mancante (ma superflua), la colonna con id topic 3. Well done! 😀

5) Mostrami per il sensore 5 tutti gli alert ✅

SELECT
    a.trigger_time,
    a.ae_value
FROM
    alert a
JOIN
    sensor s ON a.id_topic = s.id
WHERE
    s.id = 5;
